# RAG: Teaching AI Your Data

LLMs are trained on public internet data. They don't know about:
- Your company's internal documents
- Yesterday's meeting notes
- Your product manual
- Any private information

**RAG** (Retrieval-Augmented Generation) solves this. It lets your AI search your documents and use what it finds to answer questions.

```
WITHOUT RAG                              WITH RAG
──────────                               ────────

User: "What's our refund policy?"        User: "What's our refund policy?"
                                                        │
         │                                              ▼
         ▼                               ┌─────────────────────────────┐
    ┌─────────┐                          │  Search your documents...   │
    │   LLM   │                          │  Found: refund_policy.pdf   │
    └─────────┘                          └─────────────────────────────┘
         │                                              │
         ▼                                              ▼
"I don't have access to                  ┌─────────────────────────────┐
 your company's policies."               │   LLM + your document       │
                                         └─────────────────────────────┘
                                                        │
                                                        ▼
                                         "Our refund policy allows
                                          returns within 30 days..."
```

**In this course, think of RAG as a Tool** — just like Calculator or Wikipedia. Your agent can call it when it needs facts from your documents.

| | Memory | RAG |
|--|--------|-----|
| **Purpose** | Remember conversation history | Retrieve knowledge from files |
| **When used** | Every message (automatic) | When agent needs specific info |
| **Data source** | Previous messages | Your indexed documents |

RAG is not chat memory. Memory keeps track of "what did the user say earlier?" RAG answers "what do my documents say about this?"

---

## Lexical Search vs Semantic Search

There are two fundamentally different ways to search text:

| | Lexical Search | Semantic Search |
|--|----------------|-----------------|
| **How it works** | Matches exact words | Matches meaning |
| **Also called** | Keyword search, full-text search | Vector search, similarity search |
| **Strength** | Fast, precise for exact terms | Understands synonyms, context |
| **Weakness** | Misses synonyms and related concepts | Can miss exact matches (IDs, codes) |

### The Vocabulary Mismatch Problem

Lexical search has a fundamental flaw: people use different words for the same thing.

```
YOUR DOCUMENTS SAY:                    USER SEARCHES FOR:
───────────────────                    ──────────────────

"Return items within 30 days           "refund policy"
 for a full credit."
                                              │
"Contact customer care for                    │
 assistance with exchanges."                  ▼

                              ┌─────────────────────────┐
                              │    LEXICAL SEARCH       │
                              │                         │
                              │  "refund" ≠ "return"    │
                              │  "policy" ≠ "credit"    │
                              │                         │
                              │      ❌ NO MATCH        │
                              └─────────────────────────┘

The document ANSWERS the question, but the WORDS don't match.
```

This is called the **vocabulary mismatch problem**. Real examples:

| User searches for... | Document says... | Lexical finds it? |
|---------------------|------------------|-------------------|
| "laptop" | "notebook computer" | No |
| "car issues" | "vehicle problems" | No |
| "how to cancel" | "termination process" | No |
| "cheap flights" | "budget airfare" | No |

### Semantic Search Solves This

Semantic search converts text to numbers that represent **meaning**, not just words.

```
USER QUERY: "refund policy"         YOUR DOCUMENTS:
           ↓                        ───────────────
    [meaning vector]                "Return items within 30 days" → [meaning vector]
           ↓                        "Contact customer care..."    → [meaning vector]
           ↓                        "Shipping takes 3-5 days"     → [meaning vector]
           ↓
           └──────────────┐
                          ▼
              ┌─────────────────────────┐
              │   SEMANTIC SEARCH       │
              │                         │
              │  Compare meanings...    │
              │                         │
              │  ✓ "Return items..."    │  ← closest meaning
              │  ✗ "Contact customer.." │
              │  ✗ "Shipping takes..."  │
              └─────────────────────────┘
```

The words "refund" and "return" are different, but their **meanings** are close — so semantic search finds the match.

---

## Embeddings: How Semantic Search Works

How does semantic search know that "refund" and "return" are related? Through **embeddings**.

An **embedding** is a list of numbers (called a "vector") that captures the meaning of text. Think of it as translating words into coordinates on a map.

```
TEXT                              EMBEDDING (simplified)
────                              ─────────────────────
"king"                     →      [0.2, 0.8, 0.1, ...]
"queen"                    →      [0.2, 0.8, 0.3, ...]
"apple"                    →      [0.9, 0.1, 0.2, ...]
```

Notice: "king" and "queen" have similar numbers because they're related concepts. "apple" is completely different.

### Similar Meanings = Close Vectors

Imagine a map where related concepts cluster together:

```
                    ▲
                    │
         "queen" ●  │  ● "king"
                    │
        "princess" ●│● "prince"
                    │
    ────────────────┼────────────────▶
                    │
                    │      ● "apple"
                    │  ● "banana"
                    │      ● "orange"
                    │

    Royalty clusters together.    Fruits cluster together.
    Far apart from each other.
```

When you search for "monarch", the system:
1. Converts "monarch" to a vector
2. Finds vectors closest to it on the map
3. Returns "king" and "queen" — even though the word "monarch" doesn't appear anywhere

### Who Creates the Embeddings?

Embedding models (like OpenAI's `text-embedding-3-small`) are trained on massive amounts of text to learn these meaning-relationships. You don't need to understand how they work — just know that:

- Same embedding model must be used for indexing AND querying
- Different models produce different vectors (not compatible)
- Embedding API calls have a cost (but much cheaper than LLM calls)

---

## Vector Stores: A Database for Meanings

A **vector store** is a database optimized for storing and searching embeddings.

```
TRADITIONAL DATABASE                   VECTOR STORE
────────────────────                   ────────────

┌────────────────────────┐             ┌────────────────────────┐
│  ID  │  Text           │             │  ID  │  Text  │ Vector │
├──────┼─────────────────┤             ├──────┼────────┼────────┤
│  1   │  "Return items  │             │  1   │  ...   │ [0.2,  │
│      │   within 30..." │             │      │        │  0.8]  │
│  2   │  "Contact us at │             │  2   │  ...   │ [0.5,  │
│      │   support@..."  │             │      │        │  0.3]  │
└──────┴─────────────────┘             └──────┴────────┴────────┘

Search: WHERE text LIKE '%refund%'     Search: Find vectors closest to
Result: Nothing found                          [0.2, 0.7] ("refund policy")
                                       Result: Document 1 (closest match)
```

The vector store doesn't look for matching words — it finds documents whose *meaning* is closest to your query.

---

## The RAG Pipeline: Two Separate Workflows

RAG requires **two separate workflows** in n8n:

| Workflow | When to run | What it does |
|----------|-------------|--------------|
| **Indexing** | Once (or when docs change) | Load → Chunk → Embed → Store |
| **Chat/Agent** | Every user question | Embed query → Search → Answer |

This separation is important: you don't re-index your documents every time someone asks a question.

### Workflow 1: Indexing (Build Your Knowledge Base)

Run this once to prepare your documents (or schedule it to run periodically if docs change):

```
┌─────────────┐    ┌─────────────┐    ┌─────────────┐    ┌─────────────┐
│  Your Docs  │───▶│   Chunker   │───▶│  Embeddings │───▶│Vector Store │
│  (PDF, txt) │    │(split text) │    │(text→vector)│    │  (save)     │
└─────────────┘    └─────────────┘    └─────────────┘    └─────────────┘

Example:
────────
policy.pdf          "Return items      [0.2, 0.8, ...]     Stored and
(10 pages)    →     within 30 days"  →                  →  ready to
                    (100 chunks)       (100 vectors)       search
```

### Workflow 2: Chat/Agent (Answer Questions)

Run this every time a user asks something:

```
┌─────────────┐    ┌─────────────┐    ┌─────────────┐    ┌─────────────┐
│    User     │───▶│  Embeddings │───▶│Vector Store │───▶│     LLM     │
│   Question  │    │(query→vector)│   │  (search)   │    │  (answer)   │
└─────────────┘    └─────────────┘    └─────────────┘    └─────────────┘
                                             │
                                             ▼
                                      Top 3 relevant
                                         chunks

Example:
────────
"What's your         [0.2, 0.7]       "Return items       "Our refund policy
 refund policy?"  →                →  within 30 days..." → allows returns
                     (query vector)   (most similar)       within 30 days..."
```

The LLM receives your question *plus* the relevant document chunks. Now it can answer accurately using **only** the provided context.

---

## Why Chunking Matters

You might wonder: why not just send the entire document to the LLM? Three reasons:

### 1. Context Window Limits

LLMs can only process a limited amount of text at once — this is called the **context window**. Different models have different limits, but even the largest context windows can't fit hundreds of pages of documentation.

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                         LLM CONTEXT WINDOW                                  │
│                                                                             │
│    ┌───────────────────────────────────────────────────────────────────┐   │
│    │  System prompt + User question + Retrieved chunks + Response      │   │
│    └───────────────────────────────────────────────────────────────────┘   │
│                                                                             │
│    Context windows range from ~16K to ~200K tokens depending on model      │
│    One token ≈ 4 characters in English (roughly 0.75 words)                │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

If your company has 500 pages of documentation, it won't fit. You need to retrieve just the relevant parts.

### 2. Cost Efficiency

LLM pricing is based on tokens processed. More tokens = higher cost.

```
APPROACH 1: Send everything                APPROACH 2: RAG
──────────────────────────                 ────────────────

Your docs: 500 pages                       Your docs: 500 pages
                                                  │
         │                                        ▼
         ▼                                 ┌─────────────┐
┌─────────────────────┐                    │   Search    │
│  500 pages → LLM    │                    │  "refund"   │
│                     │                    └─────────────┘
│  💰 Expensive       │                           │
│  ⏱️ Slow            │                           ▼
└─────────────────────┘                    ┌─────────────┐
                                           │  3 chunks   │
                                           │  → LLM      │
                                           │             │
                                           │ 💰 Cheap    │
                                           │ ⏱️ Fast     │
                                           └─────────────┘
```

RAG is typically **much cheaper** because you only send a few relevant chunks, not hundreds of pages.

### 3. Focus and Accuracy

When LLMs receive too much text, they can get "lost" — missing important details buried in irrelevant content. Smaller, focused chunks lead to better answers.

```
                    TOO MUCH CONTEXT
                    ────────────────
┌─────────────────────────────────────────────────────────┐
│ Page 1: Company history...                              │
│ Page 2: Mission statement...                            │
│ Page 3: Team bios...                                    │
│ ...                                                     │
│ Page 47: Refund policy (30 days, receipt required)  ◀── │ Important!
│ ...                                                     │
│ Page 98: Holiday hours...                               │
└─────────────────────────────────────────────────────────┘

    LLM might miss page 47 buried in 98 pages of text.


                    FOCUSED CONTEXT (RAG)
                    ─────────────────────
┌─────────────────────────────────────────────────────────┐
│ Chunk: "Return items within 30 days with receipt for    │
│         full refund. Exchanges available within 60..."  │
└─────────────────────────────────────────────────────────┘

    LLM has exactly what it needs to answer.

---

## Chunking Strategies

How you split documents affects everything. Bad chunks = bad retrieval = bad answers.

### The Chunking Trade-off

```
CHUNK SIZE SPECTRUM
───────────────────

Too Small (50 tokens)           Just Right (300-800)           Too Large (2000+ tokens)
─────────────────────           ────────────────────           ───────────────────────

"Returns within"                "Return items within           "Return items within 30
                                 30 days with receipt           days with receipt for full
"30 days with"                   for full refund.               refund. Exchanges available
                                 Exchanges available            within 60 days. Contact
"receipt for full"               within 60 days."               customer service at... [long
                                                                paragraph about shipping,
                                                                store hours, etc.]"

❌ Lost context                  ✓ Complete thought             ❌ Too much noise
❌ Fragments don't make sense    ✓ Focused                      ❌ Irrelevant info retrieved
❌ Many chunks to search         ✓ Searchable                   ❌ Wastes context window
```

### Chunk Overlap

Overlap prevents cutting sentences in half and losing context at boundaries.

```
WITHOUT OVERLAP                         WITH OVERLAP (20%)
───────────────                         ──────────────────

Document: "Our refund policy allows returns within 30 days.
           A receipt is required for all returns."

Chunk 1: "Our refund policy allows"     Chunk 1: "Our refund policy allows
                                                  returns within 30 days."
Chunk 2: "returns within 30 days."      
                                        Chunk 2: "returns within 30 days.
Chunk 3: "A receipt is required"                  A receipt is required
                                                  for all returns."
Chunk 4: "for all returns."
                                        The overlap ensures complete
❌ "returns within 30 days" is cut       sentences are preserved.
   off from "refund policy"
```

**Typical settings:** Chunk size 500-1000 tokens, overlap 10-20%.

### What Happens with Large PDFs?

Processing a 200-page PDF:

```
┌──────────────────────────────────────────────────────────────────────────────┐
│                          200-PAGE PDF PROCESSING                             │
└──────────────────────────────────────────────────────────────────────────────┘

Step 1: LOAD                    Step 2: SPLIT                  Step 3: EMBED
────────────                    ─────────────                  ─────────────

┌──────────────┐               ┌──────────────┐               ┌──────────────┐
│   200 pages  │               │  ~400 chunks │               │ 400 vectors  │
│   (~100,000  │      →        │  (500 tokens │      →        │  stored in   │
│    words)    │               │   each)      │               │ vector store │
└──────────────┘               └──────────────┘               └──────────────┘

                  Text Splitter              Embedding Model
                  breaks into               converts each chunk
                  manageable pieces         to a vector


Step 4: QUERY
─────────────

User: "What's the refund policy?"

         │
         ▼
┌──────────────────────────────────────────────────────────────────────────────┐
│  Search 400 vectors → Return top 3-5 most relevant chunks → Send to LLM     │
└──────────────────────────────────────────────────────────────────────────────┘

Result: Only ~2,500 tokens sent to LLM (not 100,000!)
```

### Chunking Strategies Comparison

| Strategy | How it works | Best for |
|----------|--------------|----------|
| **Fixed size** | Split every N characters | Simple, predictable |
| **Recursive** | Split by paragraphs, then sentences, then words | Most documents (recommended) |
| **By page** | Each PDF page = one chunk | Documents where pages are self-contained |
| **By section** | Split at headers (##, ###) | Markdown, structured docs |

n8n's **Recursive Character Text Splitter** is the most versatile — it tries to keep paragraphs together, then sentences, then words.

---

## RAG Components in n8n

n8n provides nodes for each step of the RAG pipeline:

| Component | n8n Node | What it does |
|-----------|----------|-------------|
| **Document Loader** | Default Data Loader, GitHub, Google Drive | Reads your files |
| **Text Splitter** | Recursive Character Text Splitter | Breaks docs into chunks |
| **Embeddings** | OpenAI Embeddings, Ollama Embeddings | Converts text → vectors |
| **Vector Store** | Simple Vector Store, Supabase, Pinecone | Stores and searches vectors |

### Vector Store Options

| Option | Best for | Notes |
|--------|----------|-------|
| **Simple Vector Store** | Learning, small projects | In-memory, no setup needed |
| **Supabase** | Production with PostgreSQL | Free tier available |
| **Pinecone** | Large-scale production | Managed service |
| **Qdrant** | Self-hosted production | Open source |

For this course, we'll use **Simple Vector Store** — it works immediately without any external database setup.

> **Warning: Simple Vector Store Limitations**
>
> - **Not persistent**: Data is stored in memory and **can be erased when n8n restarts**
> - **Shared access**: All users on the same n8n instance can access the stored data
> - **Not for production**: Don't use it for confidential documents or real applications
>
> For production, use a proper vector database (Supabase, Pinecone, Qdrant).

### RAG as an Agent Tool

For agents, n8n provides the **Vector Store Question Answer Tool**. This packages RAG as a tool your agent can call:

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                              AI AGENT                                       │
│                                                                             │
│    Agent receives question → Decides to use RAG tool → Gets answer          │
│                                                                             │
│    Tools available:                                                         │
│    ├── Calculator                                                           │
│    ├── Wikipedia                                                            │
│    └── Vector Store QA Tool  ◀── Searches your indexed documents           │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

This is the recommended pattern for agents that need to answer questions from your documents.

**Docs:** [RAG in n8n](https://docs.n8n.io/advanced-ai/rag-in-n8n/) | [Vector Store Question Answer Tool](https://docs.n8n.io/integrations/builtin/cluster-nodes/sub-nodes/n8n-nodes-langchain.toolvectorstore/)

---

## RAG vs Fine-Tuning vs Long Context

There are other ways to give LLMs custom knowledge. When should you use RAG?

| Approach | How it works | Best for |
|----------|--------------|----------|
| **RAG** | Search docs at query time | Facts that change, large doc collections |
| **Fine-tuning** | Retrain the model on your data | Teaching new skills or style |
| **Long context** | Paste entire docs in the prompt | Small, static documents |

### When to Choose RAG

```
                        Use RAG when:
                        ─────────────
┌─────────────────────────────────────────────────────────────────┐
│                                                                 │
│  ✓ Documents change frequently (policies, product info)        │
│  ✓ Too many documents to fit in one prompt                     │
│  ✓ You need citations ("where did this answer come from?")     │
│  ✓ Different users need access to different documents          │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘

                        DON'T use RAG when:
                        ──────────────────
┌─────────────────────────────────────────────────────────────────┐
│                                                                 │
│  ✗ You have just 1-2 small documents (paste in prompt instead) │
│  ✗ You want to change HOW the model writes (fine-tune instead) │
│  ✗ The info is already in the model's training data            │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

## Common RAG Challenges

RAG isn't magic. Here are common issues and how to handle them:

| Challenge | What happens | Solution |
|-----------|--------------|----------|
| **Wrong chunks retrieved** | Semantic search finds unrelated text | Improve chunking, add metadata filters |
| **Answer not in docs** | LLM hallucinates an answer anyway | Add "only answer from context" instruction |
| **Chunks too small** | Missing context around the answer | Increase chunk size or overlap |
| **Chunks too large** | Irrelevant text included | Decrease chunk size |
| **Outdated embeddings** | New docs not searchable | Re-run indexing pipeline |
| **Slow queries** | Vector search takes too long | Use faster vector store, reduce chunk count |

### Hallucination Prevention

Even with RAG, LLMs may invent answers when the context doesn't contain the information. Add this to your system prompt:

```
Answer ONLY based on the provided context.
If the answer is not in the context, say "I don't have information about that."
Do not make up information.
```

### When Semantic Search Fails

Semantic search has blind spots — it can miss:

| What it misses | Example | Solution |
|----------------|---------|----------|
| **Exact IDs/codes** | "Order #12345" | Add keyword search (hybrid) |
| **Acronyms** | "What does HIPAA mean?" | Expand acronyms in docs |
| **Very specific terms** | Technical jargon | Ensure terms appear in training data |

For production systems, **hybrid search** (semantic + lexical) often works best.

---

## Summary

| Concept | Key Point |
|---------|----------|
| **RAG** | Retrieval-Augmented Generation — search your docs, then answer |
| **Lexical vs Semantic** | Keyword matching vs meaning matching |
| **Vocabulary mismatch** | Why keyword search fails ("refund" ≠ "return") |
| **Embeddings** | Convert text to vectors that capture meaning |
| **Vector Store** | Database optimized for similarity search |
| **Context Window** | LLMs can only process limited text — RAG retrieves what's relevant |
| **Chunking** | Split docs into searchable pieces (500-1000 tokens typical) |
| **Overlap** | Prevents losing context at chunk boundaries (10-20%) |

**Key insight:** RAG makes LLMs useful for your private data without retraining. Instead of stuffing everything into the prompt (expensive, slow, unfocused), you search for relevant chunks and send only those.